In [1]:
import datetime
import numpy as np

from itertools import islice, cycle
from more_itertools import pairwise

import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import seaborn as sns
sns.set(style='whitegrid')
sns.set(rc={'figure.figsize':(17, 9)})

from IPython.core.display import display, HTML, clear_output
display(HTML('<style>.container { width:80% !important; }</style>'))
display(HTML('<style>.prompt { min-width:10ex !important; }</style>'))
display(HTML('<style>div#notebook { font-size:12px !important; }</style>'))

from preprocessing import leave_last_out, transform_indices, reindex_data, generate_interactions_matrix, get_interaction_matrix
from datetime import datetime
from scipy.sparse import csr_matrix

C:\Users\Максимилиан\AppData\Local\Temp\ipykernel_24260\4098367440.py:20: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
C:\Users\Максимилиан\AppData\Local\Temp\ipykernel_24260\4098367440.py:20: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
users_df = pd.read_csv('./dataset/users_processed.csv')
items_df = pd.read_csv('./dataset/items_processed.csv')
interactions_df = pd.read_csv('./dataset/interactions_processed.csv', parse_dates=['last_watch_dt'])
#submission = pd.read_csv('sample_submission.csv')

# User features preprocessing

In [4]:
def ohe(features : list, df, items = False):
    if items:
        ohe_df = df.item_id
    else:
        ohe_df = df.user_id
    for feat in features:
        ohe_feat_df = pd.get_dummies(df[feat], prefix = feat)
        ohe_df = pd.concat([ohe_df,ohe_feat_df],axis=1)
    return ohe_df

In [56]:
# One-hot encoding of cathegorical features
user_cat_features = ['age','income','sex','kids_flg']

user_ohe_df  =  ohe(user_cat_features,users_df) #.drop(columns='user_id')

# Item features preprocessing

In [57]:
item_cat_feats = ['content_type','release_year_cat','for_kids','age_rating','studios','countries','directors']
item_ohe_df = ohe(item_cat_feats,items_df, items = True) #.drop(columns='item_id')

# Total preprocessing

In [60]:
def preprocessing(interactions_df, users_df_ohe, items_df_ohe, cold_users_split = 5, itemid = 'last_watch_dt'):
    interactions_df = interactions_df[interactions_df.user_id.isin(users_df_ohe.user_id.unique())]
    interactions_df['last_watch_dt_ts'] = interactions_df['last_watch_dt'].apply(lambda x: int(x.timestamp()))
    num_interaction_pu = interactions_df.groupby('user_id')['item_id'].count().sort_values(ascending = False)
    # get cold_users
    cold_users = num_interaction_pu.loc[(num_interaction_pu < 5) & (num_interaction_pu > 2)].index
    

    # warm_users_history
    warm_users_history = interactions_df[~interactions_df.user_id.isin(cold_users)]
    
    # cold_users_history
    cold_users_history = interactions_df[interactions_df.user_id.isin(cold_users)]
    
    # standard scenario train/holdout split
    training, holdout = leave_last_out(warm_users_history, userid='user_id', timeid=itemid)

    train_val, data_index_train = transform_indices(training, 'user_id', 'item_id')
    holdout_val = reindex_data(holdout, data_index_train, fields="items") 
    
    # cold_start_scenario train/holdout split
    training, holdout = leave_last_out(cold_users_history, userid='user_id', timeid=itemid)

    cu_val, data_index_cu = transform_indices(cold_users_history, 'user_id', 'item_id')
    cu_holdout = reindex_data(holdout, data_index_cu, fields="items") 
    
    data_description = dict(
    users = data_index_train['users'].name,
    items = data_index_train['items'].name,
    feedback = 'watched_pct',
    n_users_train = len(data_index_train['users']),
    n_items = data_index_train['items'].shape[0],   #interactions_df.item_id.nunique(),
    user_features = csr_matrix(user_ohe_df[user_ohe_df.user_id.isin(data_index_train['users'])].drop(columns='user_id').values),
    item_features = csr_matrix(item_ohe_df[item_ohe_df.item_id.isin(data_index_train['items'])].drop(columns='item_id').values),
    holdout_standard = holdout_val,
    holdout_cs = cu_holdout,
    cold_start_test = cu_val,
    )
    
    # get interactions matrix
    train_matrix, iid_to_item_id, item_id_to_iid, uid_to_user_id, user_id_to_uid = \
        get_interaction_matrix(train_val, n_items = data_description['n_items'] )
    
    train_matrix_indices = dict(
    iid_to_itemid = iid_to_item_id,
    itemid_to_iid = item_id_to_iid,
    uid_to_user_id = uid_to_user_id,
    user_id_to_uid = user_id_to_uid)
    
    # cold users
    cold_users_matrix, iid_to_item_id_cu, item_id_to_iid_cu, uid_to_user_id_cu, user_id_to_uid_cu = \
get_interaction_matrix(cold_users_history,n_items = data_description['n_items'])
    
    cold_start_matrix_indices = dict(
    iid_to_itemid = iid_to_item_id_cu,
    itemid_to_iid = item_id_to_iid_cu,
    uid_to_user_id = uid_to_user_id_cu,
    user_id_to_uid = user_id_to_uid_cu)
    
    
    return train_val,data_description, train_matrix,train_matrix_indices, cold_users_matrix, cold_start_matrix_indices
    

In [61]:
train_val,data_description, train_matrix,train_matrix_indices, cold_users_matrix, cold_start_matrix_indices = \
preprocessing(interactions_df, user_ohe_df, item_ohe_df, cold_users_split = 5, itemid = 'last_watch_dt')

C:\Users\Максимилиан\AppData\Local\Temp\ipykernel_24260\1435709166.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions_df['last_watch_dt_ts'] = interactions_df['last_watch_dt'].apply(lambda x: int(x.timestamp()))
C:\Users\Максимилиан\Desktop\Skoltech\Intro to RecSys\FP\preprocessing.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['uid'] = data[userid].astype('category')
C:\Users\Максимилиан\Desktop\Skoltech\Intro to RecSys\FP\preprocessing.py:214: SettingWithCopyWarning: 
A value is 

# Interactions filtering

# Generate interaction matrix standard scenario

## LightFM 

In [65]:
from src.baselines.LightFM.Lightfm import build_lfm_model, lightfm_scoring

In [66]:
lfm_config = dict(
    no_components = 30,
    loss = 'warp',
    max_sampled = 3,
    max_epochs = 100,
    learning_schedule = 'adagrad',
    user_alpha = 1e-3,
    item_alpha = 1e-3,
)


In [67]:
from lightfm import LightFM

In [68]:
lfm = LightFM(
        no_components=lfm_config['no_components'],
        loss=lfm_config['loss'],
        learning_schedule=lfm_config['learning_schedule'],
        # learning_rate=
        user_alpha=lfm_config['user_alpha'],
        item_alpha=lfm_config['item_alpha'],
        max_sampled=lfm_config['max_sampled'],
        # random_state =
    )

In [98]:
lfm.fit(train_matrix,
        user_features=data_description['user_features'],
        item_features=data_description['item_features'],
        epochs=20,
        verbose = True)

Epoch: 100%|██████████████████████████████████████████████████████████████████████████| 20/20 [38:35<00:00, 115.78s/it]


In [99]:
# standard scenario prediction

In [100]:
topn = 20

In [101]:
dtype = 'i4'
all_users = np.arange(data_description['n_users_train'], dtype=dtype)
test_items = np.arange(data_description['n_items']).astype(dtype)
item_index, user_index = np.meshgrid(test_items, all_users, copy=False)
all_items = np.arange(data_description['n_items'], dtype=dtype)

In [102]:
def topn_recommendations(scores, topn=10):
    recommendations = np.apply_along_axis(topidx, 1, scores, topn)
    return recommendations


def topidx(a, topn):
    parted = np.argpartition(a, -topn)[-topn:]
    return parted[np.argsort(-a[parted])]

In [103]:
score = lfm.predict(user_index[105].ravel(),item_ids = all_items.ravel(), 
             user_features=data_description['user_features'],
             item_features=data_description['item_features'],
             num_threads = 4)

In [104]:
scores = np.expand_dims(score,axis=0)
scores_topn = topn_recommendations(scores,topn)

In [105]:
scores_topn

array([[14633, 13283,  9307,  3748, 13639,  3337,   128,  8684,  2181,
         6753, 10023, 12367,  4401,   752, 12871,  3712, 10852,  3872,
        10869, 14286]], dtype=int64)

In [106]:
scores_topn

array([[14633, 13283,  9307,  3748, 13639,  3337,   128,  8684,  2181,
         6753, 10023, 12367,  4401,   752, 12871,  3712, 10852,  3872,
        10869, 14286]], dtype=int64)

In [ ]:

preds = np.zeros((all_users.shape[0],topn))

for i in range(len(all_users)):
    if i % 1000 == 0:
        print(i)
    score = lfm.predict(user_index[i].ravel(),item_ids = all_items.ravel(), 
             user_features=data_description['user_features'],
             item_features=data_description['item_features'],
             num_threads = 4)
    scores = np.expand_dims(score,axis=0)
    scores_topn = topn_recommendations(scores,topn)
    preds[i,:] = scores_topn

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


# HybridSVD

In [ ]:
d = 0.5 # off-diagonal similarity factor

# s item similarity matrix

L = np.linalg.cholesky(s)

u2,s2,v2 = np.linalg.svd(a.dot(L), full_matrices = False)

v2 = v2.T[:,:rank]

# preparing for hybrid folding-in calculation

lv = L.dot(v2)

rv = spsolve_triangular(csr_matrix(L.T),v2, lower = False)

recs2 = p.dot(lv).dot(rv.T)

recs2